In [ ]:
import json
import jq
from jaiqu import validate_schema, translate_schema

### Desired data format 

Create a `jsonschema` dictionary for the format of data you want. Data extracted from your input will be extracted into this format.

In [ ]:
schema = {
    "$schema": "http://json-schema.org/draft-07/schema#",
    "type": "object",
    "properties": {
        "id": {
            "type": ["string", "null"],
            "description": "A unique identifier for the record."
        },
        "date": {
            "type": "string",
            "description": "A string describing the date."
        },
        "model": {
            "type": "string",
            "description": "A text field representing the model used."
        }
    },
    "required": [
        "id",
        "date"
    ]
}

### Sample input data
Provoide an input JSON dictionary containing the data you want to extract.

In [ ]:
input_json = {
    "call.id": "123",
    "datetime": "2022-01-01",
    "timestamp": 1640995200,
    "Address": "123 Main St",
    "user": {
        "name": "John Doe",
        "age": 30,
        "contact": "john@email.com"
    }
}

### (Optional) Create hints
The jaiqu agent may not know certain concepts. For example, you might want to have some keys interpreted a certain way (i.e. interpret "contact" as "email"). For tricky interpretations, create hints.

In [ ]:
key_hints = "We are processing outputs of an containing an id and a date of a user."

In [ ]:
schema_properties, valid = validate_schema(input_json, schema, key_hints=key_hints, openai_api_key='key')

### Verify schema
Verify the input JSON contains the keys and values requested in your schema

In [ ]:
print('Schema is valid:',valid)
print('-'*10)
print(json.dumps(schema_properties, indent=2))

In [ ]:
jq_query = translate_schema(input_json, schema, key_hints=key_hints, max_retries=20, openai_api_key='key')

## Finalized jq query

In [ ]:
jq_query

### Check the jq query results

In [ ]:
result = jq.compile(jq_query).input(input_json).all()
print(jq.compile(jq_query).input(input_json).all())